In [9]:
!pip install faiss-cpu --no-cache-dir



In [10]:
!pip install datasets 'farm-haystack[all]'

  Using cached azure_ai_formrecognizer-3.3.3-py3-none-any.whl.metadata (64 kB)
  Using cached boto3-1.36.5-py3-none-any.whl.metadata (6.6 kB)
  Using cached elastic_transport-7.16.0-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached elasticsearch-7.17.12-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached faiss-cpu-1.7.2.tar.gz (42 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached mlflow-2.20.0-py3-none-any.whl.metadata (30 kB)
  Using cached openai_whisper-20240930-py3-none-any.whl
  Using cached opensearch_py-2.8.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached pinecone_client-2.2.4-py3-none-any.whl.metadata (7.8 kB)
  Using cached psycopg2_binary-2.9.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached pymongo-4.10.1-

## Loading corpus dataset

In [2]:
from datasets import load_dataset
from datasets import Features, Value

# Define schema
features_qrels = Features({
    "query-id": Value("string"),
    "corpus-id": Value("string"),
    "score": Value("int32")
})

qrels = load_dataset("clarin-knext/fiqa-pl-qrels", features=features_qrels)
corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")
queries = load_dataset("clarin-knext/fiqa-pl", "queries")

queries = queries["queries"]
corpus = corpus["corpus"]

queries_dict = {query["_id"]: query["text"] for query in queries}
corpus_dict = {corpus_entry["_id"]: corpus_entry["text"] for corpus_entry in corpus}
print(corpus_dict.get('3'))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.


In [5]:
qrels

DatasetDict({
    train: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 14166
    })
    validation: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1238
    })
    test: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1706
    })
})

In [6]:
corpus

Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 57638
})

In [7]:
queries

Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 6648
})

## Creating FAISSDocumentStore

In [5]:
!pip install farm-haystack[faiss]

  Using cached faiss-cpu-1.7.2.tar.gz (42 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached psycopg2_binary-2.9.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached SQLAlchemy_Utils-0.41.2-py3-none-any.whl.metadata (4.2 kB)
  Using cached SQLAlchemy-1.4.54-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
Using cached SQLAlchemy-1.4.54-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
Using cached psycopg2_binary-2.9.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
Using cached SQLAlchemy_Utils-0.41.2-py3-none-any.whl (93 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for faiss-cpu (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This 

In [4]:
from haystack.document_stores import FAISSDocumentStore
from haystack.schema import Document


document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

documents = []

for doc in corpus:
  content = f'passage: {doc["text"]}'
  documents.append(Document(content=content, id=str(doc["_id"])))



document_store.write_documents(documents, batch_size=500)


ImportError: cannot import name 'FAISSDocumentStore' from 'haystack.document_stores' (/usr/local/lib/python3.11/dist-packages/haystack/document_stores/__init__.py)

## Creating Multilingual E5 retriever

In [ ]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(document_store = document_store,
                               embedding_model="intfloat/multilingual-e5-base")

#generate embeddings for the documents
document_store.update_embeddings(retriever)

Inferencing Samples: 100%|██████████| 239/239 [03:36<00:00,  1.10 Batches/s]
Documents Processed: 60000 docs [28:46, 34.75 docs/s]


## Haystack Pipeline

In [ ]:
from haystack import Pipeline
query_pipeline = Pipeline()
query_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])

In [ ]:
result = query_pipeline.run(query = "query: Co jest wydatkiem służbowym w podróży służbowej?",
                            params={"Retriever": {"top_k": 5}})
result

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 19.24 Batches/s]


{'documents': [],
 'root_node': 'Query',
 'params': {'Retriever': {'top_k': 5}},
 'query': 'query: Co jest wydatkiem służbowym w podróży służbowej?',
 'node_id': 'Retriever'}

# Calculating NDGC@5 for the dense retriever

In [ ]:
path_to_gdrive="/content/drive/MyDrive/pjn_lab6"
%cd {path_to_gdrive}

/content/drive/MyDrive/pjn_lab6


In [ ]:
from NDCG import calculate_ndcg
import numpy as np


ndcg_scores = []
for query in queries:
  true_scores = {}
  query_text = query["text"]
  query_id = query["_id"]
  for qrel in qrels["train"]:
    if qrel["query-id"] == query_id:
      true_scores[qrel["corpus-id"]] = 1.0

  retr_scores = {}
  results = query_pipeline.run(query = query_text)
  for i, doc in enumerate(results["documents"]):
    if i == 5:
      break
    retr_scores[doc.id] = doc.score

  ndcg_scores.append(calculate_ndcg(true_scores, retr_scores))

print("NDCG@5 for Multilingual E5 retriever: ", np.mean(np.array(ndcg_scores)))


Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 19.06 Batches/s]

NDCG@5 for Multilingual E5 retriever:  0.0006939107542906826


### Summary

NDCG@5 Elastic Search (lab2): **0.17** \
NDCG@5 for Multilingual E5 retriever: **0.00069**

> Which of the methods: lexical match (e.g. ElasticSearch) or dense representation works better?

ES...

> Which of the methods is faster?

Elastic Search: ~20min \
Multilingual E5 retriever: ~1h



In [ ]:
!mv /content/faiss_document_store.db {path_to_gdrive}